# Daniel Knight's Lesson 3 Assignment
This is the Jupyter notebook for my Lesson 3 Assignment! I tried to break down each section with a header that tells what happened, but jupyter notebook styling isn't something I have spent any time doing. Please enjoy!

**Step 1:**
* Define initial import statements and input variables

In [281]:
import datetime
import os, re, sys
import numpy as np
import pandas as pd

workspace = r''
usernameAGOL = 'Insert AGOL username here'
employeeList = ['Smith, Richard', 'Madison, Margaret', 'Moore, Lisa']
startDate =  datetime.datetime(2016, 5, 19)
endDate = datetime.datetime(2017, 5, 19)
outShapefile = ''
outCsv = 'employee_travel_list.csv'

**Step 2:**
* use pandas to read the data from the employees.csv file, create titles, and verify that there is no built-in header.

In [283]:
employee_df = pd.read_csv(r'employees.csv', names =['Name', 'ID'], header=None)
employee_df

,Name,ID
0,"Smith, Richard",1234421
1,"Moore, Lisa",1231233
2,"Jones, Frank",2132222
3,"Brown, Justine",2132225
4,"Samulson, Roger",3981232
5,"Madison, Margaret",1876541


**Step 3:**
* read in all of the travel csv files and put them into a single data frame
* Here, I used a simple regex and glob.glob (inspired by many examples I found online) to find the csv files that match the required pattern.
* Finally, the data from the files were concatenated into a single data frame.

In [284]:
import glob
from datetime import datetime
filenames = glob.glob(workspace + "/travel_1*.csv")
fileList = []
for file in filenames:
    bigdf = pd.read_csv(file, names = ['ID', 'Start', 'End', 'Route'], parse_dates=[1,2], date_parser= lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'), index_col=None, header=None)
    fileList.append(bigdf)

trips_df = pd.concat(fileList, axis=0, ignore_index=True)
trips_df

,ID,Start,End,Route
0,2132222,2016-01-07 16:00:00,2016-01-26 12:00:00,Cleveland;Bangor;Erie;Philadelphia;New York;Al...
1,1234421,2016-01-15 13:00:00,2016-01-31 17:00:00,Philadelphia;Portland;Harrisburg;Burlington;Er...
2,2132225,2016-01-29 12:00:00,2016-02-03 15:00:00,Bangor;Cleveland;Augusta
3,2132225,2016-02-10 07:00:00,2016-02-22 22:00:00,Altoona;Augusta;Altoona;Bangor;Augusta;Columbus
4,2132222,2016-02-19 14:00:00,2016-02-25 13:00:00,Boston;Philadelphia
...,...,...,...,...
70,3981232,2017-12-15 07:00:00,2017-12-28 15:00:00,Harrisburg;New York;Altoona;Philadelphia;Washi...
71,2132222,2017-12-23 10:00:00,2018-01-10 20:00:00,Altoona;Scranton;Columbus;Syracuse;Pittsburgh;...
72,3981232,2017-12-29 17:00:00,2018-01-17 11:00:00,Syracuse;Augusta;Burlington;Syracuse;Pittsburg...
73,1231233,2018-01-11 07:00:00,2018-01-28 19:00:00,New York;Erie;Scranton;New York;Columbus;Bosto...


**Step 4:**
* Create new data frame from the trips_df using copy(), then use datetime operations to create a trip 'Duration' value.
* Insert Duration between ID and Start time in trips_df.
* Merge the employee_df data frame with the new_trips_df data frame, using the ID value as the key pair.
* Finally, make another copy (so I don't mess anything up with the correct frame!) and sort it by Name > Duration > Start > End, all in ascending order. That is, alphabetical by name, then shortest trips first. If two trips have the same duration, the earlier trip will be the earlier entry.

In [287]:
new_trips_df = trips_df.copy()

duration = []
for row in new_trips_df.itertuples(index=False):
    empStartDate = row[1]
    empEndDate = row[2]
    delta = empEndDate - empStartDate
    duration.append(delta.days)

new_trips_df.insert(1, 'Duration', duration)
new_trips_df

,ID,Duration,Start,End,Route
0,2132222,18,2016-01-07 16:00:00,2016-01-26 12:00:00,Cleveland;Bangor;Erie;Philadelphia;New York;Al...
1,1234421,16,2016-01-15 13:00:00,2016-01-31 17:00:00,Philadelphia;Portland;Harrisburg;Burlington;Er...
2,2132225,5,2016-01-29 12:00:00,2016-02-03 15:00:00,Bangor;Cleveland;Augusta
3,2132225,12,2016-02-10 07:00:00,2016-02-22 22:00:00,Altoona;Augusta;Altoona;Bangor;Augusta;Columbus
4,2132222,5,2016-02-19 14:00:00,2016-02-25 13:00:00,Boston;Philadelphia
...,...,...,...,...,...
70,3981232,13,2017-12-15 07:00:00,2017-12-28 15:00:00,Harrisburg;New York;Altoona;Philadelphia;Washi...
71,2132222,18,2017-12-23 10:00:00,2018-01-10 20:00:00,Altoona;Scranton;Columbus;Syracuse;Pittsburgh;...
72,3981232,18,2017-12-29 17:00:00,2018-01-17 11:00:00,Syracuse;Augusta;Burlington;Syracuse;Pittsburg...
73,1231233,17,2018-01-11 07:00:00,2018-01-28 19:00:00,New York;Erie;Scranton;New York;Columbus;Bosto...


In [288]:
merged_df = employee_df.merge(new_trips_df, left_on='ID', right_on='ID')
merged_df

,Name,ID,Duration,Start,End,Route
0,"Smith, Richard",1234421,16,2016-01-15 13:00:00,2016-01-31 17:00:00,Philadelphia;Portland;Harrisburg;Burlington;Er...
1,"Smith, Richard",1234421,7,2016-04-09 06:00:00,2016-04-16 22:00:00,Augusta;New York;Columbus;Syracuse;Albany
2,"Smith, Richard",1234421,10,2016-04-22 15:00:00,2016-05-02 20:00:00,Columbus;Burlington;Augusta;Syracuse;Augusta;P...
3,"Smith, Richard",1234421,9,2016-05-09 06:00:00,2016-05-18 22:00:00,Boston;Albany;Harrisburg;Erie
4,"Smith, Richard",1234421,6,2016-05-19 08:00:00,2016-05-25 16:00:00,Portland;Washington
...,...,...,...,...,...,...
70,"Madison, Margaret",1876541,12,2017-03-05 11:00:00,2017-03-17 14:00:00,Altoona;Columbus;Philadelphia;Augusta;Bangor;B...
71,"Madison, Margaret",1876541,14,2017-04-11 15:00:00,2017-04-25 18:00:00,Portland;Burlington;Cleveland;Altoona;Scranton...
72,"Madison, Margaret",1876541,17,2017-05-20 15:00:00,2017-06-06 17:00:00,Albany;Boston;Pittsburgh;Cleveland;New York;Ph...
73,"Madison, Margaret",1876541,11,2018-01-05 10:00:00,2018-01-16 13:00:00,Philadelphia;Burlington;Washington;Erie


In [289]:
mc = merged_df.copy()
mc = mc[(mc['Start'] > startDate) & (mc['End'] < endDate)]
mc = mc.sort_values(by=['Name', 'Duration', 'Start', 'End'], ascending=True)
mc = mc[(mc['Name'] == employeeList[0]) | (mc['Name'] == employeeList[1]) | (mc['Name'] == employeeList[2])]
mc

,Name,ID,Duration,Start,End,Route
70,"Madison, Margaret",1876541,12,2017-03-05 11:00:00,2017-03-17 14:00:00,Altoona;Columbus;Philadelphia;Augusta;Bangor;B...
71,"Madison, Margaret",1876541,14,2017-04-11 15:00:00,2017-04-25 18:00:00,Portland;Burlington;Cleveland;Altoona;Scranton...
69,"Madison, Margaret",1876541,17,2016-11-15 12:00:00,2016-12-03 11:00:00,Philadelphia;New York;Bangor;Augusta;Pittsburg...
25,"Moore, Lisa",1231233,5,2017-01-27 07:00:00,2017-02-01 18:00:00,Burlington;Bangor;Erie;Columbus
20,"Moore, Lisa",1231233,10,2016-08-22 09:00:00,2016-09-01 12:00:00,Portland;Augusta;Washington;Cleveland;Bangor;B...
23,"Moore, Lisa",1231233,10,2016-10-23 16:00:00,2016-11-02 16:00:00,Portland;Burlington;Syracuse;Bangor;Philadelphia
21,"Moore, Lisa",1231233,12,2016-09-15 11:00:00,2016-09-27 13:00:00,Augusta;Cleveland;Washington;Syracuse;Clevelan...
22,"Moore, Lisa",1231233,13,2016-09-30 14:00:00,2016-10-13 17:00:00,Augusta;Boston;Bangor;Syracuse;Portland
19,"Moore, Lisa",1231233,14,2016-06-02 18:00:00,2016-06-17 13:00:00,Portland;Harrisburg;Augusta;Columbus;Syracuse;...
24,"Moore, Lisa",1231233,20,2016-12-30 07:00:00,2017-01-19 18:00:00,Cleveland;New York;Philadelphia;New York;Augus...


**Final Output from step 4:**

In [290]:
final_df = mc.copy()
final_df

,Name,ID,Duration,Start,End,Route
70,"Madison, Margaret",1876541,12,2017-03-05 11:00:00,2017-03-17 14:00:00,Altoona;Columbus;Philadelphia;Augusta;Bangor;B...
71,"Madison, Margaret",1876541,14,2017-04-11 15:00:00,2017-04-25 18:00:00,Portland;Burlington;Cleveland;Altoona;Scranton...
69,"Madison, Margaret",1876541,17,2016-11-15 12:00:00,2016-12-03 11:00:00,Philadelphia;New York;Bangor;Augusta;Pittsburg...
25,"Moore, Lisa",1231233,5,2017-01-27 07:00:00,2017-02-01 18:00:00,Burlington;Bangor;Erie;Columbus
20,"Moore, Lisa",1231233,10,2016-08-22 09:00:00,2016-09-01 12:00:00,Portland;Augusta;Washington;Cleveland;Bangor;B...
23,"Moore, Lisa",1231233,10,2016-10-23 16:00:00,2016-11-02 16:00:00,Portland;Burlington;Syracuse;Bangor;Philadelphia
21,"Moore, Lisa",1231233,12,2016-09-15 11:00:00,2016-09-27 13:00:00,Augusta;Cleveland;Washington;Syracuse;Clevelan...
22,"Moore, Lisa",1231233,13,2016-09-30 14:00:00,2016-10-13 17:00:00,Augusta;Boston;Bangor;Syracuse;Portland
19,"Moore, Lisa",1231233,14,2016-06-02 18:00:00,2016-06-17 13:00:00,Portland;Harrisburg;Augusta;Columbus;Syracuse;...
24,"Moore, Lisa",1231233,20,2016-12-30 07:00:00,2017-01-19 18:00:00,Cleveland;New York;Philadelphia;New York;Augus...


**Step 5:**
* convert data frame to csv

In [291]:
output = workspace + '\\' + outCsv
final_df.to_csv(output)

**Step 6:**
* use Geopandas and the read_file() function to read in the ne_cities.shp into a data frame

In [292]:
import geopandas as gpd
from shapely.geometry import Point

citiesshp = workspace + '\\' + 'ne_cities.shp'
cities = gpd.read_file(citiesshp)
cities

,OBJECTID,UIDENT,POPCLASS,NAME,CAPITAL,STATEABB,COUNTRY,geometry
0,15,62507.0,2.0,Bangor,-1.0,US-ME,USA,POINT (-68.77619 44.81189)
1,20,64707.0,2.0,Waterville,-1.0,US-ME,USA,POINT (-69.62073 44.53724)
2,23,65607.0,2.0,Augusta,1.0,US-ME,USA,POINT (-69.78141 44.32166)
3,26,66707.0,2.0,Berlin,-1.0,US-NH,USA,POINT (-71.19324 44.46470)
4,29,67407.0,2.0,Lewiston,-1.0,US-ME,USA,POINT (-70.19406 44.10972)
...,...,...,...,...,...,...,...,...
132,406,121107.0,2.0,Hickory,-1.0,US-NC,USA,POINT (-81.36642 35.72144)
133,413,121907.0,3.0,Knoxville,-1.0,US-TN,USA,POINT (-83.93407 35.98019)
134,414,122007.0,2.0,Oak Ridge,-1.0,US-TN,USA,POINT (-84.25911 36.00918)
135,421,122707.0,2.0,Asheville,-1.0,US-NC,USA,POINT (-82.54221 35.60217)


**Step 7:**
* Use GDAL/OGR to create a new shapefile for every entry from the final_df data frame. 
* Combine the geometry with the final_df to create a new dataframe with LineString objects for every trip
* Use information from the dataframe to create an output shapefile with fields for the Name, Duration, and Route

In [295]:
from osgeo import gdal
from osgeo import ogr
from osgeo import osr

drv = ogr.GetDriverByName('ESRI Shapefile')
dataSet = drv.Open(citiesshp)

In [296]:
wkt = [ 'LineString (' + ','.join([ '{0} {1}'.format(cities[cities.NAME == city].geometry.x.iloc[0], cities[cities.NAME == city].geometry.y.iloc[0]) for city in r.split(';') ])+')' for r in final_df.Route]

final_geom_df = final_df.copy()
final_geom_df['geom'] = wkt
final_geom_df

,Name,ID,Duration,Start,End,Route,geom
70,"Madison, Margaret",1876541,12,2017-03-05 11:00:00,2017-03-17 14:00:00,Altoona;Columbus;Philadelphia;Augusta;Bangor;B...,LineString (-78.40098738336093 40.525651718052...
71,"Madison, Margaret",1876541,14,2017-04-11 15:00:00,2017-04-25 18:00:00,Portland;Burlington;Cleveland;Altoona;Scranton...,LineString (-70.27758484016806 43.698554552950...
69,"Madison, Margaret",1876541,17,2016-11-15 12:00:00,2016-12-03 11:00:00,Philadelphia;New York;Bangor;Augusta;Pittsburg...,LineString (-75.16727846213115 39.941228213611...
25,"Moore, Lisa",1231233,5,2017-01-27 07:00:00,2017-02-01 18:00:00,Burlington;Bangor;Erie;Columbus,LineString (-73.2096932475452 44.5032972268857...
20,"Moore, Lisa",1231233,10,2016-08-22 09:00:00,2016-09-01 12:00:00,Portland;Augusta;Washington;Cleveland;Bangor;B...,LineString (-70.27758484016806 43.698554552950...
23,"Moore, Lisa",1231233,10,2016-10-23 16:00:00,2016-11-02 16:00:00,Portland;Burlington;Syracuse;Bangor;Philadelphia,LineString (-70.27758484016806 43.698554552950...
21,"Moore, Lisa",1231233,12,2016-09-15 11:00:00,2016-09-27 13:00:00,Augusta;Cleveland;Washington;Syracuse;Clevelan...,LineString (-69.78141264717578 44.321655641294...
22,"Moore, Lisa",1231233,13,2016-09-30 14:00:00,2016-10-13 17:00:00,Augusta;Boston;Bangor;Syracuse;Portland,LineString (-69.78141264717578 44.321655641294...
19,"Moore, Lisa",1231233,14,2016-06-02 18:00:00,2016-06-17 13:00:00,Portland;Harrisburg;Augusta;Columbus;Syracuse;...,LineString (-70.27758484016806 43.698554552950...
24,"Moore, Lisa",1231233,20,2016-12-30 07:00:00,2017-01-19 18:00:00,Cleveland;New York;Philadelphia;New York;Augus...,LineString (-81.57637997965219 41.492262952473...


In [297]:
sr = osr.SpatialReference()
sr.ImportFromEPSG(4326)
outfile = drv.CreateDataSource(r'C:\GradSchool\Geog489\Leeson3\L3_Assignment\assignment3_data_March19\L3_assignment_dck5284.shp')
outlayer = outfile.CreateLayer('employeeTrips_dck5284', geom_type=ogr.wkbLineString, srs=sr) 

nameField = ogr.FieldDefn('Name', ogr.OFTString)
outlayer.CreateField(nameField)
nameField = ogr.FieldDefn('Route', ogr.OFTString)
outlayer.CreateField(nameField)
nameField = ogr.FieldDefn('Duration', ogr.OFTInteger)
outlayer.CreateField(nameField)

featureDefn = outlayer.GetLayerDefn()

for row in final_geom_df.itertuples(index=False):
    geom = ogr.CreateGeometryFromWkt(row[-1])
    outFeature = ogr.Feature(featureDefn)
    outFeature.SetGeometry(geom)
    outFeature.SetField('Name', row[0])
    outFeature.SetField('Route', row[5])
    outFeature.SetField('Duration', row[2])
    outlayer.CreateFeature(outFeature)
    outFeature = None
    
outfile = None

**Step 8:**
* Zip the output shapefile to prepare it to be publish to ArcGIS Online
* import arcgis and required methods to prepare publishing
* Search for existing publications on own account to verify no duplicates. If duplicated, it will delete the old one (i.e. running this notebook more than once
* Publish the shapefile, then publish as a feature layer
* Create a map object and widget inside Jupyter and display the data from AGOL

In [299]:
import zipfile

# function to create zipped shapefile for a given filename without extension
def zipShapefile(name):
    compiledRE = re.compile(name+'(?!.zip)\....$')
    with zipfile.ZipFile( os.path.join(workspace, name + '.zip'), 'w', zipfile.ZIP_DEFLATED) as zf:  # create zipfile
        for file in os.listdir(workspace):                                                           # go through files in workspace
            if compiledRE.match(file):                                                               # test whether file is part of the shapefile to be zipped
                zf.write(os.path.join(workspace,file),file,zipfile.ZIP_DEFLATED)                     # add file to zipfile
                
# create zipped version of observation shapefile
zipShapefile( os.path.splitext(outShapefile)[0] )

In [300]:
os.environ["PATH"] = r"{};{}".format(os.environ["PATH"], r"C:\Program Files\ArcGIS\Pro\bin") # path has to be updated if you have installed Pro in a different location
sys.path.append(r"C:\Program Files\ArcGIS\Pro\Resources\ArcPy")                              # path has to be updated if you have installed Pro in a different location

import arcgis
from arcgis.gis import GIS

# connect to AGOL using ArcGIS Pro connection
gis = GIS('pro') 
gis

GIS @ https://www.arcgis.com/ version:9.4

In [301]:
# search for content in your gis with a query built from tilte, owner and item type
def searchAGOL(title, owner, itemType):
    return gis.content.search(query='title:'+title+' owner:'+owner, item_type=itemType)

# test whether items exist on AGOL for given title, owner, and item type and if so, delete them from AGOL
def deleteIfExistsOnAGOL(title, owner, itemType):
    result = searchAGOL(title, owner, itemType)   # search item
    print('Found items for title='+title+', owner='+owner+', itemType='+itemType+':')
    print(result)
    for item in result:                           # delete items found
        item.delete()
        print('Item ' + item.title + ' has been deleted.')

deleteIfExistsOnAGOL(os.path.splitext(outShapefile)[0], usernameAGOL, 'Shapefile')
deleteIfExistsOnAGOL(os.path.splitext(outShapefile)[0], usernameAGOL, 'Feature Service')

Found items for title=L3_assignment_dck5284, owner=dck5284_pennstate, itemType=Shapefile:
[<Item title:"L3_assignment_dck5284" type:Shapefile owner:dck5284_pennstate>]
Item L3_assignment_dck5284 has been deleted.
Found items for title=L3_assignment_dck5284, owner=dck5284_pennstate, itemType=Feature Service:
[<Item title:"L3_assignment_dck5284" type:Feature Layer Collection owner:dck5284_pennstate>]
Item L3_assignment_dck5284 has been deleted.


In [302]:
print(searchAGOL(os.path.splitext(outShapefile)[0], usernameAGOL, 'Shapefile'))
print(searchAGOL(os.path.splitext(outShapefile)[0], usernameAGOL, 'Feature Service'))

[]
[]


In [303]:
tripsAGOLShapefile = gis.content.add({'type':'Shapefile'}, os.path.join(workspace, os.path.splitext(outShapefile)[0]+'.zip'))

In [304]:
tripsAGOLFeatureService = tripsAGOLShapefile.publish()

In [305]:
tripsMap = gis.map('Binghamton')
tripsMap.zoom = 5.5
tripsMap.add_layer(tripsAGOLFeatureService)
tripsMap

MapView(layout=Layout(height='400px', width='100%'))